<a href="https://colab.research.google.com/github/eyaler/avatars4all/blob/master/yarok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Video Green Screen

## Qin et al., U^2-Net: Going Deeper with Nested U-Structure for Salient Object Detection, https://arxiv.org/abs/2005.09007, https://github.com/NathanUA/U-2-Net

## Ke et al., MODNet: Is a Green Screen Really Necessary for Real-Time Portrait Matting?, https://arxiv.org/abs/2011.11961, https://github.com/ZHKKKe/MODNet

### Made just a little bit more accessible by Eyal Gruss ([@eyaler](twitter.com/eyaler) / [eyalgruss.com](https://eyalgruss.com) / eyalgruss@gmail.com)

#### Foreground options:
*   Image from web or upload
*   Video from web or upload
*   Trim video start time and duration
*   Override frame dimensions and rate
*   Concatenate multiple videos
*   Cut to short clips and mix up with options for random effects
*   Mirror versions of the above

#### Model options:
*   U^2-Net
*   U^2-Net p (small)
*   U^2-Net human segmentation
*   MODNet photographic model
*   MODNet "webcam" model
*   U^2-Net portrait generation (sketch)
*   U^2-Net portrait generation + U^2-Net blending
*   U^2-Net preprocessing + U^2-Net portrait generation
*   U^2-Net preprocessing + U^2-Net portrait generation + U^2-Net blending
*   similar combinations as above with any of the other U^2-Net models

#### Blending options:
*   Continuous blending
*   One frame delay smoothing with custom threshold
*   Grab sketch colors from original foreground
*   Foreground blend refinement with pymatting's [Fast Multi-Level Foreground Estimation](https://pymatting.github.io/pymatting.foreground.html#module-pymatting.foreground.estimate_foreground_ml) (https://arxiv.org/abs/2006.14970)

#### Background options:
*   Solid white, black, chroma green, chroma blue or any hex value
*   Transparent (for images)
*   Image from web or upload
*   Video from web or upload
*   Trim video start time and duration
*   Override frame dimensions and rate
*   Concatenate multiple videos
*   Cut to short clips and mix up with options for random effects
*   Loop video from start or in reverse
*   Resize to match foreground, optionally keeping aspect ratio
*   Foreground image/video with optional following effects
*   Bokeh (gamma-corrected blurred) versions of the above
*   Grayscale versions of the above
*   Mirror versions of the above

#### Examples:
*   U^2-Net/MODNet model comparison: https://twitter.com/eyaler/status/1342853322127110146
*   U^2-Net portrait sketching "Take On Me": https://twitter.com/eyaler/status/1346395761236443138, https://vimeo.com/showcase/8491391
*   Sexy vaporwave lady series: https://youtu.be/74vgrfIVE9A

##### A curated list of online generative tools: [j.mp/generativetools](https://j.mp/generativetools)

In [ ]:
#@title Setup
%cd /content
!git clone --depth 1 https://github.com/eyaler/U-2-Net
import os

!mkdir -p /content/U-2-Net/saved_models/u2net
%cd /content/U-2-Net/saved_models/u2net
if not os.path.exists('/content/U-2-Net/saved_models/u2net/u2net.pth'):
  !gdown https://drive.google.com/uc?id=1ao1ovG1Qtx4b7EoskHXmi2E9rp5CHLcZ -O /content/U-2-Net/saved_models/u2net/u2net.pth
!wget --no-check-certificate -nc https://eyalgruss.com/fomm/u2net.pth

!mkdir -p /content/U-2-Net/saved_models/u2netp
%cd /content/U-2-Net/saved_models/u2netp
if not os.path.exists('/content/U-2-Net/saved_models/u2netp/u2netp.pth'):
  !gdown https://drive.google.com/uc?id=1rbSTGKAE-MTxBYHd-51l2hMOQPT_7EPy -O /content/U-2-Net/saved_models/u2netp/u2netp.pth
!wget --no-check-certificate -nc https://eyalgruss.com/fomm/u2netp.pth

!mkdir -p /content/U-2-Net/saved_models/u2net_human_seg
%cd /content/U-2-Net/saved_models/u2net_human_seg
if not os.path.exists('/content/U-2-Net/saved_models/u2net_human_seg/u2net_human_seg.pth'):
  !gdown https://drive.google.com/uc?id=1-Yg0cxgrNhHP-016FPdp902BR-kSsA4P -O /content/U-2-Net/saved_models/u2net_human_seg/u2net_human_seg.pth
!wget --no-check-certificate -nc https://eyalgruss.com/fomm/u2net_human_seg.pth

!mkdir -p /content/U-2-Net/saved_models/u2net_portrait
%cd /content/U-2-Net/saved_models/u2net_portrait
if not os.path.exists('/content/U-2-Net/saved_models/u2net_portrait/u2net_portrait.pth'):
  !gdown  -O /content/U-2-Net/saved_models/u2net_portrait/u2net_portrait.pth
!wget --no-check-certificate -nc https://eyalgruss.com/fomm/u2net_portrait.pth

%cd /content
!git clone --depth 1 https://github.com/eyaler/MODNet
%cd MODNet/pretrained

if not os.path.exists('/content/MODNet/pretrained/modnet_photographic_portrait_matting.ckpt'):
  !gdown https://drive.google.com/uc?id=1mcr7ALciuAsHCpLnrtG_eop5-EYhbCmz -O /content/MODNet/pretrained/modnet_photographic_portrait_matting.ckpt
!wget --no-check-certificate -nc https://eyalgruss.com/fomm/modnet_photographic_portrait_matting.ckpt

if not os.path.exists('/content/MODNet/pretrained/modnet_webcam_portrait_matting.ckpt'):
  !gdown https://drive.google.com/uc?id=1Nf1ZxeJZJL8Qx9KadcYYyEmmlKhTADxX -O /content/MODNet/pretrained/modnet_webcam_portrait_matting.ckpt
!wget --no-check-certificate -nc https://eyalgruss.com/fomm/modnet_webcam_portrait_matting.ckpt

%cd /content
!pip install -U youtube-dl
!pip install -U imageio
!pip install -U imageio-ffmpeg
!pip install pymatting

In [ ]:
#@title Optionally mount Google Drive { run: "auto" }
mount_google_drive = False #@param {type:"boolean"}
if mount_google_drive:
  from google.colab import drive
  drive.mount('/content/drive')

In [ ]:
#@title Get the foreground image/video and background image/video from the web
#@markdown 1. You can change the URLs to your **own** stuff!
#@markdown 2. For the background you can use the drop-down menu to alternatively choose a **solid color**, **transparent** (for images only) or the **foreground** iteslf (with optional effects). You can also feed a **hex** color value (e.g. de0000 or #de0000)
#@markdown 3. Advanced: You can mannualy upload files and put in the path to the location on Colab. This path can also be to a folder containing **several videos**.
#@markdown 4. Alternatively, you can upload **local** files in the next cells

#foreground_url = 'https://www.youtube.com/watch?v=HzpzvAPj1kw' #@param {type:"string"}
#background_url = 'https://www.youtube.com/watch?v=pXpvh6eIFBk' #@param ['White', 'Black', 'Chroma Green', 'Chroma Blue', 'Transparent', 'Foreground'] {allow-input: true}

#foreground_url = 'https://www.youtube.com/watch?v=kMpnwIGDQvU' #@param {type:"string"}
#background_url = 'https://www.youtube.com/watch?v=dMvnCyznteU' #@param ['White', 'Black', 'Chroma Green', 'Chroma Blue', 'Transparent', 'Foreground'] {allow-input: true}

foreground_url = 'https://www.youtube.com/watch?v=kMpnwIGDQvU' #@param {type:"string"}
background_url = 'https://www.youtube.com/watch?v=dMvnCyznteU' #@param ['White', 'Black', 'Chroma Green', 'Chroma Blue', 'Transparent', 'Foreground'] {allow-input: true}

import os
import youtube_dl
def is_supported(url):
    if url.lower().endswith(('.png','.jpg','.jpeg','.bmp')):
      return False
    extractors = youtube_dl.extractor.gen_extractors()
    for e in extractors:
        if e.suitable(url) and e.IE_NAME != 'generic':
            return True
    return False

if foreground_url:
  !rm -f /content/foreground
  %cd /content
  if os.path.isfile(foreground_url):
    foreground_url = [os.path.abspath(foreground_url)]
  elif os.path.isdir(foreground_url):
    foreground_url = sorted(os.path.join(os.path.abspath(foreground_url),file) for file in os.listdir(foreground_url))
  else:
    if is_supported(foreground_url):
      !rm -f /content/foreground.mp4
      !youtube-dl -f 'bestvideo[ext=mp4][vcodec!*=av01]+bestaudio[ext=m4a]/mp4' '$foreground_url' --merge-output-format mp4 -o /content/foreground
      !mv /content/foreground.mp4 /content/foreground
    if not os.path.exists('/content/foreground'):
      !wget '$foreground_url' -O /content/foreground
    foreground_url = ['/content/foreground']

if '/' in background_url or '.' in background_url or '\\' in background_url:
  !rm -f /content/background
  %cd /content
  if os.path.isfile(background_url):
    background_url = [os.path.abspath(background_url)]
  elif os.path.isdir(background_url):
    background_url = sorted(os.path.join(os.path.abspath(background_url),file) for file in os.listdir(background_url))
  else:
    if is_supported(background_url):
      !rm -f /content/background.mp4
      !youtube-dl -f 'bestvideo[ext=mp4][vcodec!*=av01]+bestaudio[ext=m4a]/mp4' '$background_url' --merge-output-format mp4 -o /content/background
      !mv /content/background.mp4 /content/background
    if not os.path.exists('/content/background'):
      !wget '$background_url' -O /content/background
    background_url = ['/content/background']
else:
  background_url = background_url.lower()

In [ ]:
#@title Optionally upload foreground image/video/videos { run: "auto" }
manually_upload_foreground = False #@param {type:"boolean"}
if manually_upload_foreground:
  from google.colab import files

  %cd /content/sample_data
  try:
    uploaded = files.upload()
  except Exception as e:
    %cd /content
    raise e

  foreground_url = sorted(os.path.join(os.path.abspath(file),file) for file in uploaded)
  %cd /content

In [ ]:
#@title Optionally upload local background image/video/videos { run: "auto" }
manually_upload_background = False #@param {type:"boolean"}
if manually_upload_background:
  from google.colab import files

  %cd /content/sample_data
  try:
    uploaded = files.upload()
  except Exception as e:
    %cd /content
    raise e

  background_url = sorted(os.path.join(os.path.abspath(file),file) for file in uploaded)
  %cd /content

In [ ]:
#@title Green that screen!
#@markdown Model notes:
#@markdown 1. u2net tends to remove more unwanted parts, but may also remove desired parts of the foreground objects.
#@markdown 2. modnet tends to keep more of the desired parts and also gives a finer boundary, but may leave in more unwanted parts (which is the more useful option if you further post edit the video).
#@markdown 3. u2net_portrait will generate a sketch + if background is image/video/"Foreground" then use u2net mask of the original foreground to blend with that background.
#@markdown 4. u2net + u2net_portrait adds a preprocessing stage to remove the background before generating a sketch of the foreground.
#@markdown
#@markdown Sketching notes:
#@markdown 1. background="White" + model="u2net_portrait" -> everything becomes a sketch
#@markdown 2. background="Foreground" + model="u2net_portrait" -> sketched foreground on top of original background
#@markdown 3. background="White" + model="u2net + u2net_portrait" -> sketched foreground with white background
#@markdown 4. background="Foreground" + model="u2net + u2net_portrait" -> variation similar to (2)
#@markdown 5. background=upload result of (1) + model="u2net" -> original foreground on top of sketched background  

model = 'u2net' #@param ['u2net', 'modnet_photographic', 'modnet_webcam', 'u2net_portrait', 'u2net + u2net_portrait']
u2net_variant = 'u2net' #@param ['u2net', 'u2netp', 'u2net_human_seg']
sketch_color = 'Gray' #@param ['Gray','Foreground','Tint outline','Tint fill']
one_frame_delay = True #@param {type:"boolean"} 
one_frame_delay_threshold = 0.1 #@param {type:"slider", min:0, max:1, step:0.05}
refine_foreground_blend = False #@param {type:"boolean"}  
start_secs_foreground = 0#@param {type:"number"}
duration_secs_foreground = 60#@param {type:"number"}
start_secs_background = 0#@param {type:"number"}
duration_secs_background = 60#@param {type:"number"}
mirror_foreground = False #@param {type:"boolean"} 
mirror_background = False #@param {type:"boolean"} 
gray_background = False #@param {type:"boolean"} 
bokeh_background = False #@param {type:"boolean"} 
bokeh_prcnt = 5 #@param {type:"slider", min:1, max:50, step:1}
bokeh_gamma = 5 #@param {type:"slider", min:1, max:50, step:1}
keep_aspect_background = True #@param {type:"boolean"}
loop_reverse_background = False #@param {type:"boolean"}
override_width = 0#@param {type:"integer"}
override_height = 0#@param {type:"integer"}
override_fps = 0#@param {type:"number"}
cut_mix_min_secs_foreground = 0#@param {type:"number"}
cut_mix_max_secs_foreground = 0#@param {type:"number"}
cut_mix_tied_background = False #@param {type:"boolean"}
cut_mix_min_secs_background = 0#@param {type:"number"}
cut_mix_max_secs_background = 0#@param {type:"number"}
cut_mix_phase_secs_background = 0#@param {type:"number"}
cut_mix_double_foreground = False #@param {type:"boolean"}
cut_mix_min_secs_double = 0#@param {type:"number"}
cut_mix_max_secs_double = 0#@param {type:"number"}
cut_mix_phase_secs_double = 0#@param {type:"number"}
cut_mix_horizontal_flip_prob = 0#@param {type:"slider", min:0, max:1, step:0.05}
cut_mix_vertical_flip_prob = 0#@param {type:"slider", min:0, max:1, step:0.05}
cut_mix_temporal_flip_prob = 0#@param {type:"slider", min:0, max:1, step:0.05}
copy_audio = True #@param {type:"boolean"}

bg_mode_max_w = 1920
chroma_thresholds = [0.5,0.25]
sketch_color = sketch_color.lower()

start_secs_foreground = max(start_secs_foreground,0)
duration_secs_foreground = max(duration_secs_foreground,0)
fg_time_params = ''
if duration_secs_foreground: 
  fg_time_params = '-ss %f -t %f'%(start_secs_foreground, duration_secs_foreground)

start_secs_background = max(start_secs_background,0)
duration_secs_background = max(duration_secs_background,0)
bg_time_params = ''
if duration_secs_background: 
  bg_time_params = '-ss %f -t %f'%(start_secs_background, duration_secs_background)

override_width = max(override_width, 0)
override_height = max(override_height, 0)
override_fps = max(override_fps, 0)
cut_mix_min_secs_foreground = max(cut_mix_min_secs_foreground, 0)
cut_mix_max_secs_foreground = max(cut_mix_min_secs_foreground, cut_mix_max_secs_foreground)
cut_mix_min_secs_background = max(cut_mix_min_secs_background, 0)
cut_mix_max_secs_background = max(cut_mix_min_secs_background, cut_mix_max_secs_background)
cut_mix_min_secs_double = max(cut_mix_min_secs_double, 0)
cut_mix_max_secs_double = max(cut_mix_min_secs_double, cut_mix_max_secs_double)
if background_url=='foreground' and cut_mix_tied_background:
  cut_mix_min_secs_background = 0
  cut_mix_max_secs_background = 0
  cut_mix_phase_secs_background = 0
if not cut_mix_double_foreground:
  cut_mix_min_secs_double = 0
  cut_mix_max_secs_double = 0
  cut_mix_phase_secs_double = 0

%cd /content

fg_dirs = {
  'u2net': '/content/U-2-Net/test_data/test_images', 
  'u2netp': '/content/U-2-Net/test_data/test_images', 
  'u2net_human_seg': '/content/U-2-Net/test_data/test_human_images'
}

mask_dirs = {
  'u2net': '/content/U-2-Net/test_data/u2net_results', 
  'u2netp': '/content/U-2-Net/test_data/u2netp_results', 
  'u2net_human_seg': '/content/U-2-Net/test_data/test_human_images_results'
}

bg_dir = '/content/bg_frames'
result_dir = '/content/out_frames'
portrait_in_dir = '/content/U-2-Net/test_data/test_portrait_images/portrait_im'
portrait_out_dir = '/content/U-2-Net/test_data/test_portrait_images/portrait_results'
for k in fg_dirs:
  fg_dir = fg_dirs[k]
  mask_dir = mask_dirs[k]
  !rm -rf $fg_dir
  !mkdir -p $fg_dir
  !rm -rf $mask_dir
  !mkdir -p $mask_dir
!rm -rf $bg_dir
!mkdir -p $bg_dir
!rm -rf $result_dir
!mkdir -p $result_dir
!rm -rf $portrait_in_dir
!mkdir -p $portrait_in_dir
!rm -rf $portrait_out_dir
!mkdir -p $portrait_out_dir

fg_dir = fg_dirs[u2net_variant]
mask_dir = mask_dirs[u2net_variant]

import imageio
imageio.plugins.freeimage.download()
import cv2
import numpy as np
import os
import glob
from time import time
import io
import PIL
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from MODNet.src.models.modnet import MODNet
from pymatting import estimate_foreground_ml

import warnings
warnings.filterwarnings("ignore")
np.random.seed(0)

torch_transforms = transforms.Compose(
  [
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
  ]
)

def fix_dims(im):
    if im.ndim == 2:
        im = np.tile(im[..., None], [1, 1, 3])
    return im[...,:3]

def imread(path, file=None):
  if file is not None:
    if type(file)==int:
      file = 'frame_%05d.png'%file
    path = os.path.join(path,file)
  try:
    im = imageio.imread(path, format='PNG-FI')
  except Exception as e:
    im = imageio.imread(path)
  im = fix_dims(im)/255
  return im

def imwrite(im, path, file=None, compression=9):
  if file is not None:
    if type(file)==int:
      file = 'frame_%05d.png'%file
    path = os.path.join(path,file)
    imageio.imwrite(path, np.uint8(im*255), compression=compression, format='PNG-FI')

def safe_resize(im, w_h_size, interp):
  w, h = w_h_size
  assert w>0 or h>0
  aspect = im.shape[1]/im.shape[0]
  if w<=0:
    w = int(np.ceil(h*aspect))
  elif h<=0:
    h = int(np.ceil(w/aspect))
  return cv2.resize(im, (w,h), interpolation=interp)

def resize_crop_pad(im, size, crop_by_max_ratio=False, crop_pad_to_height=False):
  assert not crop_by_max_ratio or not crop_pad_to_height
  if im.shape[:2] == size:
    return im
  if not size[0] or size[0]==-1:
    size = (int(round(im.shape[0]/im.shape[1]*size[1])), size[1])
  elif not size[1] or size[1]==-1:
    size = (size[0], int(round(im.shape[1]/im.shape[0]*size[0])))
  if size[0]<im.shape[0] or size[1]<im.shape[1]:
    interp = cv2.INTER_AREA
  else:
    interp = cv2.INTER_CUBIC  
  if crop_by_max_ratio:
    max_ratio = max(size[0]/im.shape[0], size[1]/im.shape[1])
    im = safe_resize(im, (int(np.ceil(im.shape[1]*max_ratio)), int(np.ceil(im.shape[0]*max_ratio))), interp=interp)
    diff0 = im.shape[0]-size[0]
    diff1 = im.shape[1]-size[1]
    if diff0>0 or diff1>0:
      im = im[diff0//2:size[0]+diff0//2, diff1//2:size[1]+diff1//2]
  elif crop_pad_to_height:
    im = safe_resize(im, (-1, size[0]), interp=interp)
    diff1 = im.shape[1]-size[1]
    if diff1>0:
      im = im[:, diff1//2:size[1]+diff1//2]
    elif diff1<0:
      im = np.pad(im, pad_width=[(0,0),(-(diff1//2),diff1//2-diff1),(0,0)])
  else:
    im = safe_resize(im, size[::-1], interp=interp)
  assert im.shape[:2] == size, (im.shape[:2],size)
  return np.clip(im,0,1)

standard_size = (override_height, override_width)
def preproc_fg(im):
  if standard_size[0] or standard_size[1]:
    im = resize_crop_pad(im, (standard_size[0], standard_size[1]), crop_pad_to_height=True)
  return im

def preproc_bg(im, size, bokeh=False, gray=False, flip_x=False, flip_y=False):
  im = resize_crop_pad(im, size, crop_by_max_ratio=keep_aspect_background)
  if bokeh:
    if bokeh_gamma>1:
      im = im**bokeh_gamma
    radius = int(bokeh_prcnt/100*np.sqrt(im.shape[0]*im.shape[1])//2*2+1)
    if radius>1:
      im = cv2.GaussianBlur(im,(radius,radius),0)
    if bokeh_gamma>1:
      im **= 1/bokeh_gamma
  if gray:
    im = fix_dims(np.dot(im, [0.2989, 0.5870, 0.1140]))
  if flip_x:
    im = np.fliplr(im)
  if flip_y:
    im = np.flipud(im)
  return im

ref_size = 512
def modnet_matting(modnet, im):
  im_h, im_w = im.shape[:2]
  im_tensor = torch_transforms(im).float()
  im_tensor = im_tensor[None, :, :, :].cuda()
  
  if max(im_h, im_w) < ref_size or min(im_h, im_w) > ref_size:
    if im_w >= im_h:
      im_rh = ref_size
      im_rw = int(im_w / im_h * ref_size)
    elif im_w < im_h:
      im_rw = ref_size
      im_rh = int(im_h / im_w * ref_size)
  else:
    im_rh = im_h
    im_rw = im_w
        
  im_rw = im_rw - im_rw % 32
  im_rh = im_rh - im_rh % 32
  
  if im_h!=im_rh or im_w!=im_rw:
    im_tensor = F.interpolate(im_tensor, size=(im_rh, im_rw), mode='area')
  
  with torch.no_grad():
    _, _, matte_tensor = modnet(im_tensor, True)
  matte_tensor = F.interpolate(matte_tensor, size=(im_h, im_w), mode='area')
  matte_tensor = matte_tensor.repeat(1, 3, 1, 1)
  return matte_tensor[0].data.cpu().numpy().transpose(1, 2, 0)

def make_file_list(folder, fps, cut_mix_min_secs=0, cut_mix_max_secs=0, cut_mix_phase_secs=0):
  files = [x for x in sorted(os.listdir(folder)) if x.endswith('.png')]
  if not fps or not cut_mix_max_secs:
    return files
  cuts = []
  start = []
  frames = int(round(cut_mix_phase_secs*fps))
  start = files[:frames]
  files = files[frames:]
  while len(files):
    frames = np.random.randint(round(cut_mix_min_secs*fps),round(cut_mix_max_secs*fps)+1)
    cuts.append(files[:frames])
    if np.random.random()<cut_mix_temporal_flip_prob:
      cuts[-1] = cuts[-1][::-1]
    files = files[frames:]
  np.random.shuffle(cuts)
  return start+[file for sublist in cuts for file in sublist]

grand_start = time()
start = time()
try:
    fg_now = imread(foreground_url[0])
    imwrite(fg_now,fg_dir,0)
except Exception:
    for fg_file in foreground_url:
      vf_args = '-vf "fps=%f"'%override_fps if override_fps else ''
      j = len(glob.glob('%s/frame_*'%fg_dir))
      try:
        !ffmpeg $fg_time_params -i $fg_file $vf_args -start_number $j $fg_dir/frame_%05d.png
        if not override_fps:
          with imageio.get_reader(fg_file, format='mp4') as reader:
            override_fps = reader.get_meta_data()['fps']
      except Exception:
        pass
    fg_now = imread(fg_dir,0)
fg_now = preproc_fg(fg_now)
standard_size = fg_now.shape[:2]
fg_files = make_file_list(fg_dir, override_fps, cut_mix_min_secs_foreground, cut_mix_max_secs_foreground, max(-cut_mix_phase_secs_background,0))

prepare_time = time()-start

start = time()
have_u2_mask = False
if model.startswith('u2net') and (model!='u2net_portrait' or type(background_url)==list or background_url=='foreground'):
  %cd /content/U-2-Net
  if u2net_variant == 'u2net_human_seg':
    !python /content/U-2-Net/u2net_human_seg_test.py
  else:
    !python /content/U-2-Net/u2net_test.py $u2net_variant
  have_u2_mask = True
elif model.startswith('modnet'):
  %cd /content/MODNet
  modnet = MODNet(backbone_pretrained=False)
  modnet = nn.DataParallel(modnet).cuda()
  modnet.load_state_dict(torch.load(os.path.join('/content/MODNet/pretrained',model+'_portrait_matting.ckpt')))
  modnet.eval()
mask_time = time()-start

blend_time = 0
is_fg = True
bg_files = []
iter_files = fg_files
rounds = 0

def extract_file_index(file):
  return int(file.rsplit('_',1)[1].split('.')[0])

def mat(in_dir, out_dir, bg_mode, orig_dir=None, is_pre=False):
  global prepare_time, blend_time, fg_now, is_fg, bg_files, iter_files, rounds, override_fps
  rounds += 1
  start = time()
  if bg_mode == 'white':
    bg = np.full_like(fg_now, 1)
  elif bg_mode == 'black':
    bg = np.full_like(fg_now, 0)
  elif bg_mode == 'chroma green':
    bg = np.full_like(fg_now, [0,177/255,64/255])
  elif bg_mode == 'chroma blue':
    bg = np.full_like(fg_now, [0,71/255,187/255])
  elif type(bg_mode)==list:
    try:
      bg = imread(bg_mode[0])
      bg = preproc_bg(bg, fg_now.shape[:2], bokeh=bokeh_background, gray=gray_background, flip_x=mirror_background, flip_y=False)
    except Exception:
      for bg_file in bg_mode:
        vf_args = '-vf "fps=%f"'%override_fps if override_fps else ''
        j = len(glob.glob('%s/frame_*'%bg_dir))
        try:
          !ffmpeg $bg_time_params -i $bg_file $vf_args -start_number $j $bg_dir/frame_%05d.png
          if not override_fps:
            with imageio.get_reader(bg_file, format='mp4') as reader:
              override_fps = reader.get_meta_data()['fps']
        except Exception:
          pass
  elif bg_mode not in ['transparent','foreground']:
    bg = np.full_like(fg_now, [int(bg_mode.lstrip('#')[i:i+2], 16)/255 for i in [0, 2, 4]])
  use_bg_dir = bg_dir
  if bg_mode=='foreground' and len(fg_files)>1 and not cut_mix_tied_background and (cut_mix_max_secs_foreground or cut_mix_max_secs_background):
    use_bg_dir = fg_dir
  bg_files = make_file_list(use_bg_dir, override_fps, cut_mix_min_secs_background, cut_mix_max_secs_background, max(cut_mix_phase_secs_background,0))
  prepare_time += time()-start

  start = time()
  fg_plus = None
  mask_plus = None
  orig = None
  is_fg = len(fg_files)>1 or not bg_files
  if is_fg:
    iter_files = fg_files
  else:
    iter_files = bg_files
    fg_now = imread(in_dir,0)
    fg_now = preproc_fg(fg_now)
    if have_u2_mask:
      mask = imread(mask_dir,0)
      mask = preproc_fg(mask)
    elif model.startswith('modnet'):
      mask = modnet_matting(modnet, fg_now)
    else:
      mask = np.ones_like(fg_now)
    if orig_dir is not None:
      orig = imread(orig_dir,0)
      orig = preproc_fg(orig)
    if mirror_foreground: #this will only happen on last pass
      fg_now = np.fliplr(fg_now)
      mask = np.fliplr(mask)
      if orig is not None:
        orig = np.fliplr(orig)

  j = -1
  j_direction = 1
  for i,file in enumerate(iter_files):
      iter_file_index = extract_file_index(iter_files[i])
      first = i==0 or abs(extract_file_index(iter_files[i-1])-iter_file_index)>1
      last = i==len(iter_files)-1 or abs(extract_file_index(iter_files[i-1])-iter_file_index)>1
      first_bg = first
      if is_fg:
        if one_frame_delay and not first and not last:
          fg_now = fg_plus
          mask_minus = mask_now
          mask_now = mask_plus
          if fg_now is None:
            fg_now = imread(in_dir,file)
            fg_now = preproc_fg(fg_now)
          if have_u2_mask:
            if mask_now is None:
              mask_now = imread(mask_dir,file)
              mask_now = preproc_fg(mask_now)
            mask_plus = imread(mask_dir,fg_files[i+1])
            mask_plus = preproc_fg(mask_plus)
          elif model.startswith('modnet'):
            if mask_now is None:
              mask_now = modnet_matting(modnet, fg_now)
            fg_plus = imread(in_dir,fg_files[i+1])
            fg_plus = preproc_fg(fg_plus)
            mask_plus = modnet_matting(modnet, fg_plus)
          else:
            if mask_now is None:
              mask_now = np.ones_like(fg_now)
            mask_plus = np.ones_like(fg_now)          
          cond = (np.abs(mask_plus-mask_minus)<=one_frame_delay_threshold) & (np.abs(mask_now-mask_minus)>one_frame_delay_threshold) & (np.abs(mask_now-mask_plus)>one_frame_delay_threshold)
          mask = mask_now*(1-cond) + (mask_minus+mask_plus)/2*cond
        else:
          fg_now = imread(in_dir,file)
          fg_now = preproc_fg(fg_now)
          if have_u2_mask:
            mask_now = imread(mask_dir,file)
            mask_now = preproc_fg(mask_now)
          elif model.startswith('modnet'):
            mask_now = modnet_matting(modnet, fg_now)
          else:
            mask_now = np.ones_like(fg_now)
          mask = mask_now

        if orig_dir is not None:  
          orig = imread(orig_dir,file)
          orig = preproc_fg(orig)

        if first:
          flip_x_fg = mirror_foreground
          flip_y_fg = False
          if cut_mix_max_secs_foreground and len(iter_files)>cut_mix_min_secs_foreground*override_fps:
            if np.random.random()<cut_mix_horizontal_flip_prob:
              flip_x_fg = not flip_x_fg
            if np.random.random()<cut_mix_vertical_flip_prob:
              flip_y_fg = not flip_y_fg

        if flip_x_fg and not is_pre:
            fg_now = np.fliplr(fg_now)
            mask = np.fliplr(mask)
            if orig is not None:
              orig = np.fliplr(orig)

        if flip_y_fg and not is_pre:
            fg_now = np.flipud(fg_now)
            mask = np.flipud(mask)
            if orig is not None:
              orig = np.flipud(orig)

        if bg_mode=='foreground' and (len(fg_files)==1 or cut_mix_tied_background or not cut_mix_max_secs_foreground and not cut_mix_max_secs_background):
          if orig is not None:
            bg = orig
          else:
            bg = fg_now
        elif bg_files:
          if loop_reverse_background: 
            j += j_direction
            if j>=len(bg_files):
              j = 2*len(bg_files)-j-1
              j_direction = -1
            elif j<0:
              j = 0
              j_direction = 1
          else:
            j = i%len(bg_files)
          bg = imread(use_bg_dir,bg_files[j])
          bg_file_index = extract_file_index(bg_files[j])
          if j_direction>0:
            first_bg = j==0 or abs(extract_file_index(bg_files[j-1])-bg_file_index)>1
          else:
            first_bg = j==len(bg_files)-1 or abs(extract_file_index(bg_files[j+1])-bg_file_index)>1
      
      else:
        bg = imread(bg_dir,file)

      if bg_files:
        if first_bg:
          flip_x_bg = mirror_background
          flip_y_bg = False
          if cut_mix_max_secs_background and len(bg_files)>cut_mix_min_secs_background*override_fps:
            if np.random.random()<cut_mix_horizontal_flip_prob:
              flip_x_bg = not flip_x_bg
            if np.random.random()<cut_mix_vertical_flip_prob:
              flip_y_bg = not flip_y_bg
        bg = preproc_bg(bg, fg_now.shape[:2], bokeh=bokeh_background, gray=gray_background, flip_x=flip_x_bg, flip_y=flip_y_bg)

      fg = fg_now
      if refine_foreground_blend:
        try:
          fg = estimate_foreground_ml(fg, mask[...,0])
        except Exception as e:
          pass
          #print(e)
          #imwrite(fg, '/content/debug_fg.png', compression=1)
          #imwrite(mask, '/content/debug_mask.png', compression=1)

      if bg_mode == 'transparent':
        im = np.dstack([fg,mask[...,0]])
      else:
        if orig is not None:
          if sketch_color=='foreground':
            fg = 1-(1-fg)*(1-orig)
          elif 'tint' in sketch_color:
            non_black_mask = np.any(mask != [0, 0, 0], axis=-1)
            colors = (orig*mask)[non_black_mask]
            chroma = colors.max(axis=-1)-colors.min(axis=-1)
            chroma_thresholds.sort(reverse=True)
            if sketch_color=='tint fill':
              chroma_thresholds.append(0)
            else:
              color = np.array([0,0,0])
            for threshold in chroma_thresholds:
              cond = chroma>=threshold
              if np.any(cond):
                unique, counts = np.unique(colors[cond], axis=0, return_counts=True)
                color = unique[np.argmax(counts)]
                break
            if sketch_color=='tint outline':
              fg = 1-(1-fg)*(1-color)
            elif sketch_color=='tint fill':
              fg = fg*color
        im = bg*(1-mask)+fg*mask
      if not override_width and not override_height and is_fg and bg_mode_max_w and fg_now.shape[1]>bg_mode_max_w: # prevent creation of too high resolutions videos due to a high resolution foreground image
        im = resize_crop_pad(im, (-1,bg_mode_max_w))
      imwrite(im, out_dir, file if is_pre else i, compression=1 if len(fg_files)>1 or bg_files else 9)
      print('%d/%d (%d)'%(i+1,len(iter_files),rounds))
  blend_time += time()-start

out_dir = result_dir
bg_mode = background_url
if 'u2net_portrait' in model:
  out_dir = portrait_in_dir
  bg_mode = 'white' 
if model!='u2net_portrait':
  mat(fg_dir, out_dir, bg_mode, is_pre='+' in model)
elif 'u2net_portrait' in model:
  if background_url=='foreground' or sketch_color!='gray':
    !cp $fg_dir/* $portrait_in_dir
  else:
    !mv $fg_dir/* $portrait_in_dir
if 'u2net_portrait' in model:
  %cd /content/U-2-Net
  start = time()
  !python /content/U-2-Net/u2net_portrait_test.py
  blend_time += time()-start
  if type(background_url)==list or background_url=='foreground' or sketch_color!='gray':
    mat(portrait_out_dir, result_dir, background_url, orig_dir=fg_dir if background_url=='foreground' or sketch_color!='gray' else None)
  else:
    !mv $portrait_out_dir/* $result_dir

start = time()
from IPython.display import HTML, clear_output, Image
from base64 import b64encode
import shutil
!rm -f /content/final.mp4
!rm -f /content/final.png
if len(fg_files)>1 or len(bg_files)>1:
  if is_fg:
    if copy_audio:
      !ffmpeg -framerate $override_fps -i $result_dir/frame_%05d.png $fg_time_params -i /content/foreground -c:v libx264 -c:a aac -map 0:v -map 1:a? -vf "crop=trunc(iw/2)*2:trunc(ih/2)*2" -pix_fmt yuv420p -profile:v baseline -movflags +faststart /content/final.mp4 -y
    else:
      !ffmpeg -framerate $override_fps -i $result_dir/frame_%05d.png -c:v libx264 -vf "crop=trunc(iw/2)*2:trunc(ih/2)*2" -pix_fmt yuv420p -profile:v baseline -movflags +faststart /content/final.mp4 -y
  else:
    if copy_audio:
      with open('/content/list.txt','w') as f:
        if bg_time_params:
          start_seconds = float(bg_time_params.split(' ')[1])
          end_seconds = start_seconds + float(bg_time_params.split(' ')[3])
        for file in background_url:
          f.write("file '%s'\n"%file)
          if bg_time_params:
            f.write('inpoint %f\n'%start_seconds)
            f.write('outpoint %f\n'%end_seconds)
      !ffmpeg -f concat -safe 0 -i /content/list.txt -c copy /content/bg_audio.mp4 -y
      !ffmpeg -framerate $override_fps -i $result_dir/frame_%05d.png -i /content/bg_audio.mp4 -c:v libx264 -c:a aac -map 0:v -map 1:a? -vf "crop=trunc(iw/2)*2:trunc(ih/2)*2" -pix_fmt yuv420p -profile:v baseline -movflags +faststart /content/final.mp4 -y
    else:
      !ffmpeg -framerate $override_fps -i $result_dir/frame_%05d.png -c:v libx264 -c:a aac -vf "crop=trunc(iw/2)*2:trunc(ih/2)*2" -pix_fmt yuv420p -profile:v baseline -movflags +faststart /content/final.mp4 -y
  #video can be downloaded from /content/final.mp4
  save_time = time()-start
  total_time = time()-grand_start  
  clear_output()
  with open('/content/final.mp4', 'rb') as f:
    data_url = "data:video/mp4;base64," + b64encode(f.read()).decode()
  display(HTML("""
  <video width=600 controls autoplay loop>
        <source src="%s" type="video/mp4">
  </video>""" % data_url))
else:
  shutil.move(os.path.join(out_dir,'frame_%05d.png'%1), '/content/final.png')
  #image can be downloaded from /content/final.png
  save_time = time()-start
  total_time = time()-grand_start
  clear_output()
  display(Image('/content/final.png', width=600))
if model.startswith('u2net'):
    print('frames=%d prepare=%d mask=%d blend=%d save=%d total=%d'%(len(iter_files), prepare_time, mask_time, blend_time, save_time, total_time))
else:
    print('frames=%d prepare=%d mask+blend=%d save=%d total=%d'%(len(iter_files), prepare_time, mask_time+blend_time, save_time, total_time))


In [ ]:
#@title Download
from google.colab import files
if os.path.exists('/content/final.mp4'):
  files.download('/content/final.mp4')
else:
  files.download('/content/final.png')